# Análisis Titanic

Notebook con el mismo **guion por bloques y ejercicios** que tu script, separado en celdas.

> Nota: Mantengo tu estructura y enunciados. Solo he corregido un detalle para que el output sea el esperado: `df.describe()` en vez de imprimir la referencia al método.

## Carga del dataset

In [ ]:

import pandas as pd
import numpy as np


In [ ]:

# Instalación (si estás en Colab/Jupyter y no lo tienes instalado)
!pip install -q ydata-profiling


In [ ]:

from ydata_profiling import ProfileReport

url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)
df.head()


## BLOQUE 1: Exploración inicial (Tiempo estimado: 20 minutos)

### Ejercicio 1. Primera inspección

- Responde:
  1) ¿Cuántas filas y columnas tiene? (shape)
  2) ¿Cuáles son los tipos de datos de cada columna? (dtypes / info)
  3) ¿Cuánta memoria ocupa el DataFrame? (info(memory_usage="deep") y/o df.memory_usage(deep=True).sum())

In [ ]:

print("1) Shape (filas, columnas):", df.shape)
print("\n2) Tipos de datos por columna (dtypes):")
print(df.dtypes)

print("\n3) Info con memoria (deep):")
df.info(memory_usage="deep")


### Ejercicio 2. Estadísticas descriptivas

- Genera un resumen estadístico de las variables numéricas. (describe)
- Responde:
  1) ¿Cuál es la edad media? (Age mean)
  2) ¿Y la tarifa media? (Fare mean)
  3) ¿Hay algún valor que te llame la atención como posible anomalía?
     (pistas: mínimos/máximos raros, medias muy altas, etc.)

In [ ]:

print("Resumen estadístico (numéricas):")
print(df.describe(), "\n")

print("1) Edad media (Age mean):", round(df['Age'].mean(), 2))
print("2) Tarifa media (Fare mean):", round(df['Fare'].mean(), 2))

print("\n3) Rango y media por variable numérica (min / max / mean):")
for col in df.select_dtypes(include='number').columns:
    print(col, "-", round(df[col].min(), 2), round(df[col].max(), 2), round(df[col].mean(), 2))


### Ejercicio 3. Distribución de variables categóricas

- Usa value_counts() para responder:
  1) ¿Cuántos pasajeros hay de cada sexo? (Sex)
  2) ¿Cuántos en cada clase? (Pclass)
  3) ¿Desde qué puerto embarcaron más pasajeros? (Embarked)

In [ ]:

print("1) Pasajeros por sexo (Sex):")
print(df['Sex'].value_counts(), "\n")

print("2) Pasajeros por clase (Pclass):")
print(df['Pclass'].value_counts(), "\n")

print("3) Pasajeros por puerto (Embarked):")
print(df['Embarked'].value_counts(), "\n")


## BLOQUE 2: Diagnóstico de calidad de datos (Tiempo estimado: 25 min)

### Ejercicio 4. Mapa de valores faltantes

- Crea un informe (DataFrame) que muestre para cada columna:
  1) Número de valores faltantes
  2) Porcentaje sobre el total
  3) Tipo de dato (dtype)
  4) Datatype completo

- Ordena el resultado de mayor a menor porcentaje de missing e informe completo.

In [ ]:

df_missing_values = pd.DataFrame({
    'col': df.columns,
    'missing_values': df.isnull().sum()
})
print("1) Missing values (conteo):")
print(df_missing_values, "\n")

df_percent = pd.DataFrame({
    'col': df.columns,
    'missing_values': round(df.isnull().mean() * 100, 2)
})
print("2) Missing values (%):")
print(df_percent, "\n")

df_ptype = pd.DataFrame({
    'col': df.columns,
    'missing_values': round(df.isnull().mean() * 100, 2),
    'data_type': df.dtypes
})
print("3) Missing + dtype:")
print(df_ptype, "\n")

df_missing_values_complete = pd.DataFrame({
    'col': df.columns,
    'missing_values': df.isnull().sum(),
    'missing_values_percent': round(df.isnull().mean() * 100, 2),
    'data_type': df.dtypes
})
print("4) Informe completo:")
print(df_missing_values_complete, "\n")

df_complete_sorted = df_missing_values_complete.sort_values(
    by='missing_values_percent', ascending=False
)
print("Informe completo ordenado (desc):")
print(df_complete_sorted)


> Si quieres un profiling del **dataset original** (más útil), pásale `df` al ProfileReport. Aquí mantengo tu idea y genero profiling del informe ordenado.

In [ ]:

profile = ProfileReport(df_complete_sorted, title="Análisis de Missing Values")
profile.to_notebook_iframe()


### Ejercicio 5. Análisis de la variable Cabin

- La columna Cabin tiene muchos faltantes.
- Para los registros que SÍ tienen cabina:
  1) Extrae la letra inicial (cubierta) -> por ejemplo "C85" -> "C"
  2) Analiza si hay relación entre la cubierta (Deck) y la clase (Pclass).
     (pistas: value_counts, crosstab, groupby, proporciones por clase)

In [ ]:

df_cabin = df[df['Cabin'].notna()].copy()
print("Registros con Cabin NO nulo:")
print(df_cabin, "\n")

df_cabin['Deck'] = df_cabin['Cabin'].str[0]
print("Cabin + Deck (primera letra):")
print(df_cabin, "\n")

print("Crosstab Deck vs Pclass (conteos):")
print(pd.crosstab(df_cabin['Deck'], df_cabin['Pclass']))


### Ejercicio 6. Detección de duplicados

- Comprueba si hay filas completamente duplicadas en el dataset.
- Comprueba duplicados parciales: mismo nombre (Name) y ticket (Ticket).
- Si los hay, plantea cómo los gestionarías.

In [ ]:

df_not_unique = df.duplicated()
print("Duplicados completos (nº filas):", df_not_unique.sum())

df_partial_not_unique = df.duplicated(subset=['Name', 'Ticket'], keep=False)
print("Duplicados parciales (Name+Ticket) (nº filas):", df_partial_not_unique.sum())

df_without_dupes = df.drop_duplicates(keep='first')
print("Shape tras drop_duplicates:", df_without_dupes.shape)


### Ejercicio 7. Consistencia de datos

- Verifica que los valores de Pclass estén en el rango esperado: {1, 2, 3}.
- Busca:
  1) Pasajeros con edad negativa (Age < 0)
  2) Tarifas negativas (Fare < 0)
- Outliers extremos en Fare.

In [ ]:

df_Pclass_valid = df[df['Pclass'].isin([1, 2, 3])]
print("Pclass válidas (filas):", len(df_Pclass_valid))

df_Pclass_invalid = df[~df['Pclass'].isin([1, 2, 3])]
print("Pclass inválidas (filas):", len(df_Pclass_invalid))

df_negative_age = df[df['Age'] < 0]
print("Edades negativas (filas):", len(df_negative_age))

df_negative_fare = df[df['Fare'] < 0]
print("Fares negativos (filas):", len(df_negative_fare))

print("\nDescribe Fare (outliers):")
print(df['Fare'].describe())


## BLOQUE 3: Tratamiento de valores faltantes (Tiempo estimado: 30 min)

### Ejercicio 8. Imputación de Age

- Extrae títulos del nombre (Mr, Mrs, Miss, Master, etc.).
- Imputa los valores faltantes de Age con la **mediana** por título.
- Justifica por qué la mediana es mejor que la media en este caso.

In [ ]:

df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
print("Columna Title (primeras filas):")
print(df['Title'].head(), "\n")

median_title = df.groupby('Title')['Age'].median()
df['Age'] = df['Age'].fillna(df['Title'].map(median_title))

print("Missing tras imputar Age:")
print(df.isnull().sum(), "\n")

mean_title = df.groupby('Title')['Age'].mean()
print("Media por Title (Age):")
print(mean_title, "\n")
print("Máxima media:", mean_title.max())
print("Mínima media:", mean_title.min(), "\n")

print("Mediana por Title (Age):")
print(median_title, "\n")
print("Máxima mediana:", median_title.max())
print("Mínima mediana:", median_title.min(), "\n")


### Ejercicio 9. Decisión sobre Cabin

- Con ~77% de valores faltantes:
  ¿tiene sentido intentar imputar Cabin?
- Propón dos alternativas:
  (a) Crear una variable binaria HasCabin (1 si Cabin no es NaN, 0 si es NaN)
  (b) Eliminar la columna Cabin
- Argumenta cuál elegirías y por qué (impacto en modelos, información útil, ruido, complejidad)

In [ ]:

# (a) Crear HasCabin y (opcional) eliminar Cabin
# df['HasCabin'] = df['Cabin'].notna().astype(int)
# df = df.drop(columns='Cabin')

# (b) Eliminar directamente Cabin
# df = df.drop(columns='Cabin')


### Ejercicio 10. Imputación de Embarked

- Solo hay 2 valores faltantes en Embarked:
  1) Identifica esos pasajeros (filas con Embarked NaN)
  2) Investiga si comparten características (Pclass, Fare, Cabin, Sex...)
  3) Imputa con el valor más probable

In [ ]:

df_Embarked_null = df[~df['Embarked'].isnull() == False]
print("Filas con Embarked NaN:")
print(df_Embarked_null, "\n")

mode_global = df['Embarked'].mode().iloc[0]
df['Embarked'] = df['Embarked'].fillna(mode_global)

print("Missing tras imputar Embarked:")
print(df.isnull().sum(), "\n")


## BLOQUE 4: Análisis exploratorio de supervivencia (Tiempo estimado: 30 min)

### Ejercicio 11. Supervivencia por sexo

- Calcula la tasa de supervivencia por sexo (Sex) y exprésalo como porcentaje.

### Ejercicio 12. Supervivencia por clase

- Calcula la tasa de supervivencia por clase (Pclass) y exprésalo como porcentaje.

### Ejercicio 13. Tabla cruzada sexo-clase

- Crea una tabla cruzada (crosstab) con la tasa de supervivencia combinando Sex y Pclass.

### Ejercicio 14. Efecto de la edad

- Crea grupos de edad: niño (0-12), adolescente (13-17), adulto (18-59), mayor (60+)

### Ejercicio 15. Efecto del tamaño familiar

- Crea FamilySize y agrupa: solo (1), pequeña (2-4), grande (5+)

In [ ]:

survivability_by_sex = df.groupby('Sex')['Survived'].mean() * 100
print("Ej.11 - Supervivencia por sexo:")
print(round(survivability_by_sex, 2).astype(str) + '%', "\n")

survivability_by_Pclass = df.groupby('Pclass')['Survived'].mean() * 100
print("Ej.12 - Supervivencia por clase:")
print(round(survivability_by_Pclass, 2).astype(str) + '%', "\n")

tabla = pd.crosstab(df['Sex'], df['Pclass'], values=df['Survived'], aggfunc='mean') * 100
print("Ej.13 - Tasa de supervivencia (%) por Sex y Pclass:")
print(tabla.round(2), "\n")
print("Interpretación (tu nota): la de tercera", "\n")

bins = [0, 12, 17, 59, df['Age'].max()]
labels = ['niño', 'adolescente', 'adulto', 'mayor']
df['Age_Group'] = pd.cut(df['Age'], bins=bins, labels=labels, include_lowest=True)

survivability_by_age_group = df.groupby('Age_Group')['Survived'].mean() * 100
print("Ej.14 - Supervivencia por grupo de edad:")
print(survivability_by_age_group, "\n")
print("Tu nota:", "chi", "\n")

df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
bins = [0, 1, 4, df['FamilySize'].max()]
labels = ['solo', 'pequeña', 'grande']
df['FamilySize'] = pd.cut(df['FamilySize'], bins=bins, labels=labels, include_lowest=True)

print("Ej.15 - FamilySize categorizada (primeras filas):")
print(df['FamilySize'].head(), "\n")

survivability_by_family_size = df.groupby('FamilySize')['Survived'].mean() * 100
print("Ej.15 - Supervivencia por tamaño familiar:")
print(survivability_by_family_size)


## BLOQUE 5: Ejercicio integrador (Tiempo estimado: 35 min)

### Ejercicio 16. Pipeline de limpieza completo

- Crea una función `limpiar_titanic(df)` que devuelva un DataFrame limpio aplicando:
  - Extraer título del nombre
  - Imputar Age con mediana por título
  - Crear HasCabin
  - Imputar Embarked con la moda
  - Crear FamilySize
  - Crear Age_Group con categorías
  - Eliminar: PassengerId, Name, Ticket, Cabin

- La función debe ser reutilizable para aplicar lo mismo al test.

In [ ]:

def limpiar_titanic(df):
    df = df.copy()

    # • Extraer título del nombre
    df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)

    # • Imputar Age con la mediana por título
    median_title = df.groupby('Title')['Age'].median()
    df['Age'] = df['Age'].fillna(df['Title'].map(median_title))

    # • Crear variable HasCabin a partir de Cabin
    df['HasCabin'] = df['Cabin'].notna().astype(int)

    # • Imputar Embarked con el valor más frecuente
    mode_global = df['Embarked'].mode().iloc[0]
    df['Embarked'] = df['Embarked'].fillna(mode_global)

    # • Crear variable FamilySize
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

    # • Crear variable AgeGroup con las categorías definidas
    bins = [0, 12, 17, 59, df['Age'].max()]
    labels = ['niño', 'adolescente', 'adulto', 'mayor']
    df['Age_Group'] = pd.cut(df['Age'], bins=bins, labels=labels, include_lowest=True)

    # • Eliminar columnas que no se usarán
    df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

    return df

df_limpio = limpiar_titanic(df)
print("Head df_limpio:")
print(df_limpio.head(), "\n")

print("Missing df_limpio:")
print(df_limpio.isnull().sum())
